# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [ ]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

import optuna
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

### 데이터 읽어오기


In [ ]:
ROOT_DIR = "data"
RANDOM_STATE = 110

# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Unit Time_Dam,CURE END POSITION X Judge Value_Dam,...,Production Qty Collect Result_Fill2,Production Qty Unit Time_Fill2,Production Qty Judge Value_Fill2,Receip No Collect Result_Fill2,Receip No Unit Time_Fill2,Receip No Judge Value_Fill2,WorkMode Collect Result_Fill2,WorkMode Unit Time_Fill2,WorkMode Judge Value_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,NaN,NaN,...,7,NaN,NaN,127,NaN,NaN,1,NaN,NaN,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,NaN,NaN,...,185,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,NaN,NaN,...,10,NaN,NaN,73,NaN,NaN,1,NaN,NaN,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,NaN,NaN,...,268,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,NaN,NaN,...,121,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,NaN,NaN,...,318,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,NaN,NaN,...,14,NaN,NaN,197,NaN,NaN,1,NaN,NaN,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,NaN,NaN,...,1,NaN,NaN,27,NaN,NaN,1,NaN,NaN,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,NaN,NaN,...,117,NaN,NaN,1,NaN,NaN,0,NaN,NaN,Normal


In [ ]:
#결측값 확인 및 삭제 : 칼럼 178로 줄임
missing_columns = train_data.columns[train_data.isnull().any()]
columns_to_drop = list(missing_columns)
train_data = train_data.drop(columns = columns_to_drop)
train_data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.200,1.0,318,1,0,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,20.5,14,197,1,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,19.7,1,27,1,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,20.100,13.0,117,1,0,Normal


In [ ]:
#이상점 칼럼 찾기 z-score 이용 + 드랍하기
#z-score : https://modulabs.co.kr/blog/outlier-detection/

#이상치
#이상치 판단 기준 설정: 일반적으로 표준적인 z-score 기준을 사용하여 이상치를 판단합니다.
#보통 z-score가 특정 기준값(일반적으로 2 ~ 3)을 넘어가는 데이터를 이상치

#이상치 처리 방법에는 다양한 방법이 있겠지만 일단은 칼럼수가 매우 많기 때문에 파라미터를 줄이기 위해 이상치가 존재하면 칼럼을 드랍해봄

def find_columns_with_outliers(dataframe, threshold=3):
    columns_with_outliers = []
    for col in dataframe.columns:
        if dataframe[col].dtype in [np.int64, np.float64]:
            z_scores = np.abs((dataframe[col] - dataframe[col].mean()) / dataframe[col].std())
            if any(z_scores > threshold):
                columns_with_outliers.append(col)
    return columns_with_outliers

columns_with_outliers = find_columns_with_outliers(train_data)
print(columns_with_outliers)

train_data = train_data.drop(columns=columns_with_outliers)
train_data

#178 - #136

['CURE SPEED Collect Result_Dam', 'Dispense Volume(Stage1) Collect Result_Dam', 'Machine Tact time Collect Result_Dam', 'PalletID Collect Result_Dam', 'Production Qty Collect Result_Dam', 'Receip No Collect Result_Dam', 'Stage1 Line2 Distance Speed Collect Result_Dam', 'Stage3 Circle1 Distance Speed Collect Result_Dam', 'THICKNESS 2 Collect Result_Dam', 'THICKNESS 3 Collect Result_Dam', '1st Pressure Collect Result_AutoClave', '1st Pressure 1st Pressure Unit Time_AutoClave', '2nd Pressure Collect Result_AutoClave', '3rd Pressure Collect Result_AutoClave', '3rd Pressure Unit Time_AutoClave', 'Chamber Temp. Collect Result_AutoClave', 'Chamber Temp. Unit Time_AutoClave', 'DISCHARGED SPEED OF RESIN Collect Result_Fill1', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill1', 'DISCHARGED TIME OF RESIN(Stage2) Collect Result_Fill1', 'DISCHARGED TIME OF RESIN(Stage3) Collect Result_Fill1', 'Dispense Volume(Stage1) Collect Result_Fill1', 'Dispense Volume(Stage2) Collect Result_Fill1', 'Dispe

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,HEAD Standby Position X Collect Result_Fill2,HEAD Standby Position Y Collect Result_Fill2,HEAD Standby Position Z Collect Result_Fill2,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3J1XF434-2,1,OK,240.0,2.5,-90,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
40502,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1XC796-1,1,OK,1000.0,12.5,90,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
40503,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,4C1XD438-1,1,OK,240.0,2.5,-90,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
40504,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3I1XA258-1,1,OK,1000.0,12.5,90,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal


In [ ]:
#100% 인 애만 드랍
#value_counts() 의 경우 빈도수로 기본적으로 내림차순, 즉 가장 빈도수 높은 값이 앞으로 나와있기 때문에 iloc[0] 을 쓰면 가장 앞에 값이 나옴
def drop_highly_uniform_columns(train_data, threshold=1):
    n_rows = len(train_data)
    uniform_columns = [col for col in train_data.columns if train_data[col].value_counts().iloc[0] / n_rows >= threshold]
    return uniform_columns

# 데이터프레임에서 99.999%가 같은 값인 칼럼을 드롭
dropped_columns = drop_highly_uniform_columns(train_data, threshold=0.99999)

# 결과 출력
print("Dropped Columns:", dropped_columns)
#결과로 나오는 칼럼은 값이 거의 다 같은 row 로 이루어져 있기 때문에 드랍해도 됨

Dropped Columns: ['Wip Line_Dam', 'Process Desc._Dam', 'Insp. Seq No._Dam', 'Insp Judge Code_Dam', 'CURE STANDBY POSITION X Collect Result_Dam', 'CURE STANDBY POSITION Z Collect Result_Dam', 'CURE STANDBY POSITION Θ Collect Result_Dam', 'CURE START POSITION Z Collect Result_Dam', 'Wip Line_AutoClave', 'Process Desc._AutoClave', 'Equipment_AutoClave', 'Insp. Seq No._AutoClave', 'Insp Judge Code_AutoClave', '1st Pressure Judge Value_AutoClave', '2nd Pressure Judge Value_AutoClave', '3rd Pressure Judge Value_AutoClave', 'Wip Line_Fill1', 'Process Desc._Fill1', 'Insp. Seq No._Fill1', 'Insp Judge Code_Fill1', 'Wip Line_Fill2', 'Process Desc._Fill2', 'Insp. Seq No._Fill2', 'Insp Judge Code_Fill2', 'CURE END POSITION Θ Collect Result_Fill2', 'CURE STANDBY POSITION X Collect Result_Fill2', 'CURE STANDBY POSITION Θ Collect Result_Fill2', 'CURE START POSITION Θ Collect Result_Fill2', 'DISCHARGED SPEED OF RESIN Collect Result_Fill2', 'DISCHARGED TIME OF RESIN(Stage1) Collect Result_Fill2', 'DISCH

In [ ]:
train_data = train_data.drop(columns=dropped_columns)
train_data

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,HEAD Standby Position X Collect Result_Fill2,HEAD Standby Position Y Collect Result_Fill2,HEAD Standby Position Z Collect Result_Fill2,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,target
0,Dam dispenser #1,AJX75334505,4F1XA938-1,240.0,2.5,-90,1030,-90,16,14.9,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
1,Dam dispenser #1,AJX75334505,3KPM0016-2,240.0,2.5,-90,1030,-90,10,21.3,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
2,Dam dispenser #2,AJX75334501,4E1X9167-1,1000.0,12.5,90,280,90,16,14.7,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
3,Dam dispenser #2,AJX75334501,3K1X0057-1,1000.0,12.5,90,280,90,10,21.3,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
4,Dam dispenser #1,AJX75334501,3HPM0007-1,240.0,2.5,-90,1030,-90,10,9.7,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,Dam dispenser #1,AJX75334501,3J1XF434-2,240.0,2.5,-90,1030,-90,10,17.0,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
40502,Dam dispenser #2,AJX75334501,4E1XC796-1,1000.0,12.5,90,280,90,16,14.9,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
40503,Dam dispenser #1,AJX75334501,4C1XD438-1,240.0,2.5,-90,1030,-90,16,14.2,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
40504,Dam dispenser #2,AJX75334501,3I1XA258-1,1000.0,12.5,90,280,90,10,9.7,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal


In [ ]:
# 문자열 패턴을 기반으로 "dispenser #1"을 1로, "dispenser #2"를 2로 변경

drop_columns2 = [
    'Chamber Temp. Judge Value_AutoClave',
    'Model.Suffix_Fill1',
    'Model.Suffix_Dam',
    'Equipment_Dam',
    'Model.Suffix_Fill2',
    'Workorder_AutoClave',
    'Workorder_Fill1',
    'Equipment_Fill1',
    'Equipment_Fill2',
    'Model.Suffix_AutoClave',
    'Workorder_Dam',
    'Workorder_Fill2'
]

# LabelEncoder 생성
label_encoder = LabelEncoder()

# 각 열에 대해 Label Encoding을 적용하여 새로운 열을 추가하고, 기존 열을 삭제
for col in drop_columns2:
    train_data[col] = label_encoder.fit_transform(train_data[col])

# 기존 칼럼을 삭제
train_data
#최종적으로 칼럼 98.

,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE START POSITION X Collect Result_Dam,CURE START POSITION Θ Collect Result_Dam,DISCHARGED SPEED OF RESIN Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage1) Collect Result_Dam,...,HEAD Standby Position X Collect Result_Fill2,HEAD Standby Position Y Collect Result_Fill2,HEAD Standby Position Z Collect Result_Fill2,Head Clean Position X Collect Result_Fill2,Head Clean Position Y Collect Result_Fill2,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,target
0,0,3,657,240.0,2.5,-90,1030,-90,16,14.9,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
1,0,3,283,240.0,2.5,-90,1030,-90,10,21.3,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
2,1,0,589,1000.0,12.5,90,280,90,16,14.7,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
3,1,0,251,1000.0,12.5,90,280,90,10,21.3,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
4,0,0,142,240.0,2.5,-90,1030,-90,10,9.7,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,0,0,238,240.0,2.5,-90,1030,-90,10,17.0,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal
40502,1,0,643,1000.0,12.5,90,280,90,16,14.9,...,243.5,270,50,-10,119,50.0,91.8,270,50,Normal
40503,0,0,540,240.0,2.5,-90,1030,-90,16,14.2,...,243.7,270,50,-10,119,50.0,91.8,270,50,Normal
40504,1,0,164,1000.0,12.5,90,280,90,10,9.7,...,270.0,50,-10,119,50,91.8,270.0,50,85,Normal


In [ ]:
top_20 = [
    "Workorder_AutoClave",
    "Workorder_Dam",
    "Workorder_Fill1",
    "Workorder_Fill2",
    "Machine Tact time Collect Result_Fill1",
    "Dispense Volume(Stage3) Collect Result_Dam",
    "DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam",
    "HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam",
    "Dispense Volume(Stage2) Collect Result_Dam",
    "HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam",
    "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1",
    "WorkMode Collect Result_Dam",
    "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1",
    "Stage1 Circle1 Distance Speed Collect Result_Dam",
    "HEAD Standby Position X Collect Result_Dam",
    "HEAD Standby Position X Collect Result_Fill1",
    "HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1",
    "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1",
    "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam",
    "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2"
]


train_data = pd.concat([train_data[top_20],z], axis=1)
train_data

,Workorder_AutoClave,Workorder_Dam,Workorder_Fill1,Workorder_Fill2,Machine Tact time Collect Result_Fill1,Dispense Volume(Stage3) Collect Result_Dam,DISCHARGED TIME OF RESIN(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam,Dispense Volume(Stage2) Collect Result_Dam,HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam,...,WorkMode Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1,Stage1 Circle1 Distance Speed Collect Result_Dam,HEAD Standby Position X Collect Result_Dam,HEAD Standby Position X Collect Result_Fill1,HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1,HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam,HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2,target
0,657,657,657,657,128.0,1.02,8.4,550.5,0.58,464.3,...,0.000,430.3,1,274.51,244.520,838.4,157.0,384.00,428.0,Normal
1,283,283,283,283,56.5,1.49,4.9,463.8,0.34,160.8,...,7.000,244.4,4000,257.00,289.000,458.5,430.5,282.15,243.7,Normal
2,589,589,589,589,128.0,1.61,8.5,162.4,0.93,465.4,...,0.003,1323.1,1,274.33,244.415,837.9,156.1,1271.80,1324.2,Normal
3,251,251,251,251,57.7,1.49,8.4,465.5,0.58,552.0,...,7.000,244.3,4000,257.00,289.000,458.8,1323.2,282.15,243.5,Normal
4,142,142,142,142,89.9,0.67,4.9,462.5,0.34,159.5,...,7.000,244.4,9000,257.00,289.000,458.5,430.2,284.80,243.7,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,238,238,238,238,51.8,1.19,5.0,463.8,0.35,160.8,...,7.000,244.4,5000,257.00,289.000,458.5,430.5,282.15,243.7,Normal
40502,643,643,643,643,128.0,1.02,8.5,163.5,0.59,466.2,...,-0.118,1324.3,1,274.33,244.535,838.1,156.1,1281.20,1324.2,Normal
40503,540,540,540,540,128.0,1.00,8.2,550.6,0.57,463.9,...,0.000,431.1,1,274.78,244.543,838.4,157.0,378.00,428.0,Normal
40504,164,164,164,164,57.4,0.67,4.9,465.1,0.34,551.3,...,7.000,244.4,9000,257.00,289.000,458.8,1323.2,282.15,243.5,Normal


### 오버 샘플링


데이타 불균형을 해결하기 위해 오버 샘플링을 진행합니다.


In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. 데이터 준비 (train_data에서 target 분리)
X = train_data.drop(columns=['target'])  # Features (target 칼럼 제외)
y = train_data['target']  # Target (Normal, AbNormal)

# 2. LabelEncoder로 라벨 인코딩 (Normal=0, AbNormal=1)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # LabelEncoding (Normal -> 0, AbNormal -> 1)

# 3. SMOTE 적용 (1:1 비율로 오버샘플링)
smote = SMOTE(sampling_strategy='minority', random_state=RANDOM_STATE)
X_smote, y_smote = smote.fit_resample(X, y_encoded)

# 4. 원래 라벨로 다시 변환
y_smote = label_encoder.inverse_transform(y_smote)

# 5. 오버샘플링된 데이터의 새로운 데이터프레임 생성
df_concat = pd.concat([pd.DataFrame(X_smote, columns=X.columns), pd.DataFrame(y_smote, columns=['target'])], axis=1)

# 6. 오버샘플링 후 데이터 분포 확인
print(df_concat['target'].value_counts())

target
Normal      38156
AbNormal    38156
Name: count, dtype: int64


### Pycaret 적용


In [ ]:
from pycaret.classification import *

exp_clf = setup(data = df_concat, target = 'target', session_id=123)

,Description,Value
0,Session id,123
1,Target,target
2,Target type,Binary
3,Target mapping,"AbNormal: 0, Normal: 1"
4,Original data shape,"(76312, 21)"
5,Transformed data shape,"(76312, 21)"
6,Transformed train set shape,"(53418, 21)"
7,Transformed test set shape,"(22894, 21)"
8,Numeric features,20
9,Preprocess,True


In [ ]:
rf = create_model('rf')
tuned_rf = tune_model(rf, optimize = 'F1')
tuned_rf

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8656,0.9460,0.8656,0.8658,0.8656,0.7312,0.7314
1,0.8641,0.9444,0.8641,0.8642,0.8641,0.7282,0.7283
2,0.8736,0.9491,0.8736,0.8739,0.8736,0.7473,0.7475
3,0.8641,0.9436,0.8641,0.8644,0.8641,0.7282,0.7285
4,0.8686,0.9461,0.8686,0.8689,0.8686,0.7372,0.7375
5,0.8620,0.9419,0.8620,0.8623,0.8620,0.7241,0.7243
6,0.8693,0.9461,0.8693,0.8696,0.8693,0.7387,0.7389
7,0.8656,0.9428,0.8656,0.8661,0.8655,0.7312,0.7317
8,0.8704,0.9511,0.8704,0.8706,0.8704,0.7409,0.7410


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6075,0.6570,0.6075,0.6150,0.6009,0.2149,0.2223
1,0.6282,0.6729,0.6282,0.6367,0.6224,0.2565,0.2648
2,0.6189,0.6679,0.6189,0.6257,0.6136,0.2377,0.2445
3,0.6149,0.6623,0.6149,0.6228,0.6087,0.2299,0.2376
4,0.6200,0.6681,0.6200,0.6283,0.6138,0.2400,0.2481
5,0.6256,0.6728,0.6256,0.6350,0.6190,0.2512,0.2605
6,0.6219,0.6723,0.6219,0.6314,0.6149,0.2437,0.2531
7,0.6217,0.6739,0.6217,0.6293,0.6160,0.2434,0.2508
8,0.6166,0.6662,0.6166,0.6270,0.6085,0.2331,0.2433


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

In [ ]:
dt = create_model('dt')
tuned_dt = tune_model(dt, optimize = 'F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8682,0.9390,0.8682,0.8682,0.8682,0.7364,0.7364
1,0.8622,0.9351,0.8622,0.8622,0.8622,0.7244,0.7244
2,0.8729,0.9405,0.8729,0.8729,0.8729,0.7458,0.7458
3,0.8609,0.9345,0.8609,0.8609,0.8609,0.7218,0.7218
4,0.8652,0.9379,0.8652,0.8653,0.8652,0.7304,0.7305
5,0.8641,0.9350,0.8641,0.8641,0.8641,0.7282,0.7282
6,0.8688,0.9401,0.8688,0.8688,0.8688,0.7376,0.7376
7,0.8673,0.9354,0.8673,0.8675,0.8673,0.7346,0.7348
8,0.8691,0.9417,0.8691,0.8692,0.8691,0.7383,0.7383


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6876,0.7812,0.6876,0.6886,0.6872,0.3751,0.3761
1,0.7055,0.7966,0.7055,0.7058,0.7055,0.4111,0.4113
2,0.7516,0.8497,0.7516,0.7518,0.7515,0.5032,0.5034
3,0.6969,0.7747,0.6969,0.6991,0.6961,0.3939,0.3960
4,0.7070,0.7933,0.7070,0.7115,0.7055,0.4141,0.4185
5,0.7211,0.8117,0.7211,0.7277,0.7190,0.4422,0.4487
6,0.7407,0.8311,0.7407,0.7413,0.7406,0.4815,0.4821
7,0.7164,0.8112,0.7164,0.7169,0.7162,0.4328,0.4333
8,0.7450,0.8418,0.7450,0.7454,0.7449,0.4900,0.4904


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
et = create_model('et')
tuned_eet = tune_model(et, optimize = 'F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8615,0.9376,0.8615,0.8615,0.8615,0.7230,0.7230
1,0.8607,0.9393,0.8607,0.8607,0.8607,0.7215,0.7215
2,0.8706,0.9450,0.8706,0.8708,0.8706,0.7413,0.7414
3,0.8579,0.9369,0.8579,0.8580,0.8579,0.7158,0.7159
4,0.8632,0.9419,0.8632,0.8632,0.8632,0.7263,0.7264
5,0.8602,0.9375,0.8602,0.8603,0.8602,0.7203,0.7205
6,0.8643,0.9414,0.8643,0.8643,0.8643,0.7286,0.7286
7,0.8617,0.9362,0.8617,0.8621,0.8616,0.7233,0.7238
8,0.8691,0.9475,0.8691,0.8692,0.8691,0.7383,0.7383


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6308,0.6851,0.6308,0.6361,0.6273,0.2617,0.2669
1,0.6471,0.6905,0.6471,0.6509,0.6449,0.2943,0.2980
2,0.6396,0.6962,0.6396,0.6429,0.6376,0.2793,0.2826
3,0.6359,0.6893,0.6359,0.6398,0.6334,0.2718,0.2757
4,0.6393,0.6987,0.6393,0.6435,0.6366,0.2785,0.2828
5,0.6473,0.6945,0.6473,0.6517,0.6448,0.2946,0.2990
6,0.6471,0.6997,0.6471,0.6525,0.6440,0.2943,0.2996
7,0.6421,0.6946,0.6421,0.6465,0.6394,0.2842,0.2885
8,0.6390,0.6875,0.6390,0.6451,0.6352,0.2780,0.2840


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [ ]:
catboost = create_model('catboost')
tuned_cat = tune_model(catboost, optimize = 'F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8604,0.9330,0.8604,0.8627,0.8601,0.7207,0.7231
1,0.8497,0.9296,0.8497,0.8521,0.8494,0.6994,0.7018
2,0.8611,0.9367,0.8611,0.8647,0.8608,0.7222,0.7258
3,0.8444,0.9261,0.8444,0.8467,0.8442,0.6889,0.6912
4,0.8484,0.9271,0.8484,0.8522,0.8480,0.6967,0.7005
5,0.8519,0.9309,0.8519,0.8553,0.8516,0.7039,0.7072
6,0.8469,0.9312,0.8469,0.8527,0.8462,0.6937,0.6995
7,0.8534,0.9302,0.8534,0.8576,0.8530,0.7069,0.7110
8,0.8585,0.9350,0.8585,0.8620,0.8581,0.7169,0.7204


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8686,0.9372,0.8686,0.8701,0.8685,0.7372,0.7387
1,0.8534,0.9348,0.8534,0.8546,0.8533,0.7069,0.7080
2,0.8677,0.9419,0.8677,0.8695,0.8675,0.7353,0.7372
3,0.8531,0.9315,0.8531,0.8551,0.8528,0.7061,0.7082
4,0.8519,0.9327,0.8519,0.8549,0.8516,0.7039,0.7068
5,0.8564,0.9376,0.8564,0.8594,0.8561,0.7128,0.7158
6,0.8574,0.9346,0.8574,0.8610,0.8570,0.7147,0.7183
7,0.8574,0.9344,0.8574,0.8596,0.8571,0.7147,0.7169
8,0.8629,0.9389,0.8629,0.8649,0.8628,0.7259,0.7278


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
blender_4= blend_models(estimator_list= [tuned_rf, tuned_eet, tuned_eet,tuned_cat])

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8710,0.9501,0.8710,0.8717,0.8710,0.7420,0.7427
1,0.8677,0.9482,0.8677,0.8679,0.8676,0.7353,0.7355
2,0.8755,0.9543,0.8755,0.8762,0.8755,0.7510,0.7518
3,0.8656,0.9474,0.8656,0.8661,0.8655,0.7312,0.7317
4,0.8678,0.9494,0.8678,0.8692,0.8677,0.7357,0.7371
5,0.8662,0.9473,0.8662,0.8668,0.8661,0.7323,0.7329
6,0.8706,0.9509,0.8706,0.8713,0.8706,0.7413,0.7420
7,0.8690,0.9481,0.8690,0.8701,0.8689,0.7379,0.7391
8,0.8753,0.9552,0.8753,0.8758,0.8753,0.7506,0.7511


## 3. 모델 학습


### 모델 정의


In [ ]:
final_model = finalize_model(blender_4)

In [ ]:
# Train/Test Split
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.2,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)

# 통계 함수 개선
def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}, " +
          f"Ratio: {num_abnormal / num_normal:.2f}")

# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 30525, AbNormal: 30524, Ratio: 1.00
  Total: Normal: 7631, AbNormal: 7632, Ratio: 1.00


### 모델 학습


In [ ]:
features = []

for col in df_train.columns:
    try:
        df_train[col] = df_train[col].astype(int)
        features.append(col)
    except:
        continue

train_x = df_train[features]
train_y = df_train["target"]

In [ ]:
final_model.fit(train_x, train_y)

Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['Workorder_AutoClave',
                                             'Workorder_Dam', 'Workorder_Fill1',
                                             'Workorder_Fill2',
                                             'Machine Tact time Collect '
                                             'Result_Fill1',
                                             'Dispense Volume(Stage3) Coll...
                                                                    min_impurity_decrease=0.0,
                                                                    min_samples_leaf=1,
                                                                    min_samples_split=2,
                                                                    min_weight_fraction_leaf=0.0,
                                                                    monotonic_cst=None,
                                                                    n_estimators=100,
                                                                    n_jobs=-1,
                                                                    oob_score=False,
                                                                    random_state=123,
                                                                    verbose=0,
                                                                    warm_start=False)),
                                              ('CatBoost Classifier',
                                               <catboost.core.CatBoostClassifier object at 0x7fc5d01d25c0>)],
                                  flatten_transform=True, n_jobs=-1,
                                  verbose=False, voting='soft',
                                  weights=None))],
         verbose=False)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [ ]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))

df_test_x = test_data[features]
drop_columns = ['Workorder_Dam', 'Workorder_AutoClave', 'Workorder_Fill1', 'Workorder_Fill2']

# LabelEncoder 생성
label_encoder = LabelEncoder()

# 각 열에 대해 Label Encoding을 적용하여 새로운 열을 추가하고, 기존 열을 삭제
for col in drop_columns:
    df_test_x[col] = label_encoder.fit_transform(df_test_x[col])

In [ ]:
for col in df_test_x.columns:
    try:
        df_test_x.loc[:, col] = df_test_x[col].astype(int)
    except:
        continue

In [ ]:
test_pred = final_model.predict(df_test_x)
test_pred = test_pred.tolist()
test_pred

['AbNormal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'AbNormal',
 'Normal',
 'Normal',
 'Normal',
 'Normal',
 'No

### 제출 파일 작성


In [ ]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["target"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
